In [1]:
import sys
sys.path.append("..")
import os

In [2]:
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
from torchvision.utils import make_grid
import torchvision.models as models
import torchvision.transforms as transforms
from vasync.models.pretext_model import ClassifierPretextModel
from vasync.utils.common import load_config
from PIL import  Image

In [3]:
def load_model(config_path, checkpoint_path):
    config = load_config(config_path)
    pretext_base_model = config["pretext_base_model"]
    num_cls = config["num_cls"]
    model = ClassifierPretextModel(pretext_base_model=pretext_base_model, 
                                   num_cls=num_cls)
    state_dict = torch.load(checkpoint_path)
    model.load_state_dict(state_dict)
    return model

In [19]:
checkpoint_path = "../outputs/img_cls/img_cls_pseudolbl_30_res18/checkpoint_epoch_211.pt"
config_path = "../experiments/img_cls_pseudolbl_30_res18/config.yml"
model = load_model(config_path, checkpoint_path)


In [20]:
feature_extractor = nn.Sequential(*list(model.children())[:-1])

In [21]:
transform = transforms.Compose([transforms.Resize(250),
                                transforms.RandomCrop(245),
                                transforms.ToTensor()])

In [30]:
image_path = "image.jpg"
img = Image.open(image_path)
img = transform(img)
# img = torch.randn(3, 245, 245)

In [31]:
out_cls = model(img.unsqueeze(0))
out_feat = feature_extractor(img.unsqueeze(0))
softmax = nn.Softmax()

In [32]:
cls = torch.argmax(softmax(out_cls.squeeze(0)))
print(cls)

tensor(8, grad_fn=<NotImplemented>)


/home/hhemati/anaconda3/envs/p37/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  """Entry point for launching an IPython kernel.


In [33]:
print(out_feat.view(-1))
print(torch.mean(out_feat))
print(torch.std(out_feat))

tensor([0.9577, 0.8805, 0.8058, 0.9287, 0.7888, 0.7403, 0.7728, 0.8075, 0.7891,
        0.9216, 0.9136, 0.8617, 0.8608, 0.8035, 0.7510, 0.9360, 0.7173, 0.7624,
        0.8416, 0.8263, 0.9226, 0.9489, 0.8930, 0.8997, 0.9084, 1.0693, 0.8130,
        0.9748, 0.8615, 0.7219, 0.8499, 0.8022, 0.8825, 0.8353, 0.8481, 0.9434,
        0.8202, 0.7612, 0.8156, 0.9212, 0.9211, 0.7550, 0.8337, 0.9041, 0.9076,
        0.7733, 0.8722, 0.8664, 0.9151, 0.7610, 0.9193, 0.8521, 0.8440, 0.9862,
        0.8516, 0.9805, 0.7971, 0.8028, 0.8070, 0.8627, 0.8286, 0.7863, 0.8303,
        0.8762, 0.8962, 0.8476, 0.8102, 0.8963, 0.8405, 0.8502, 0.8108, 0.7919,
        0.8089, 0.9208, 0.9359, 0.7912, 0.9826, 0.9035, 0.8362, 0.8499, 0.9196,
        0.8627, 0.7768, 0.7548, 0.8328, 0.8057, 0.8684, 0.8084, 0.8209, 0.8549,
        0.8819, 0.8388, 0.7871, 0.9497, 0.6854, 0.8203, 0.8782, 0.9492, 0.7442,
        0.8078, 0.8194, 0.8410, 0.8746, 0.8036, 0.8146, 0.8361, 0.8135, 0.7670,
        0.9162, 0.8360, 0.8197, 0.9117, 